In [1]:
!pip install -q wandb torchmetrics

     |████████████████████████████████| 1.8 MB 7.5 MB/s 
     |████████████████████████████████| 398 kB 52.9 MB/s 
     |████████████████████████████████| 181 kB 50.9 MB/s 
     |████████████████████████████████| 144 kB 46.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [2]:
import pickle
import random
import gzip
import pytz
from datetime import datetime
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Preprocessing Embeddings

In [3]:
compound_embedding_path = "/content/drive/MyDrive/LangOn/semanticsNTU/tencent_compounds_embeddings.txt.gz"

vocabs = []
embs = []

with gzip.open(compound_embedding_path, "rt") as fin:
  n_vocab, n_hdim = fin.readline().split(" ")
  n_hdim = int(n_hdim)
  for i in tqdm(range(int(n_vocab))):
    toks = fin.readline().strip().split(" ")
    word = toks[0]
    emb = np.array([float(x) for x in toks[1:]])
    vocabs.append(word)
    embs.append(emb)
embs = embs / np.linalg.norm(embs, axis=1)[:, np.newaxis]

  0%|          | 0/119797 [00:00<?, ?it/s]

In [4]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
sum(1 for x in vocabs if len(x)==2), sum(1 for x in vocabs if len(x)==4)

(43394, 76398)

In [9]:
class ConvmorphNNDataset(Dataset):
  def __init__(self, vocabs, embs):
    self.build_dataset(vocabs, embs)

  def build_dataset(self, vocabs, embs):
    vocab_map = {vocab: idx for idx, vocab in enumerate(vocabs)}
    self.data = []
    for idx in tqdm(range(len(vocabs)), desc="building dataset"):
      word = vocabs[idx]
      const1 = word[:2]
      const2 = word[2:]
      if len(word) < 4: continue
      if not (const1 in vocab_map and const2 in vocab_map):
        continue
      emb = embs[idx]
      const1_vec = embs[vocab_map[const2]]
      const2_vec = embs[vocab_map[const1]]      
      self.data.append(dict(
          word=word,
          const1=const1, const2=const2,
          word_vec=emb,
          const1_vec=const1_vec, const2_vec=const2_vec
      ))
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [10]:
cm_ds = ConvmorphNNDataset(vocabs, embs)
len(cm_ds)

building dataset:   0%|          | 0/119797 [00:00<?, ?it/s]

75957

In [12]:
class ConvmorphDatasetArc(Dataset):
  def __init__(self, cm_dataset, idxs):
    self.build_dataset(cm_dataset, idxs)

  def build_dataset(self, ds, idxs):
    self.data = []
    for serial, idx in enumerate(idxs):
      data_x = ds[idx]
      inputX = np.concatenate([
          data_x["const1_vec"], data_x["const2_vec"]
      ]).reshape(1, 20, 20)
      target = data_x["word_vec"]
      self.data.append(dict(
          word_id=serial,
          inputX=torch.tensor(inputX, dtype=torch.float32),
          target=torch.tensor(target, dtype=torch.float32)
      ))
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]


In [15]:
N = len(cm_ds)
rng = np.random.RandomState(123)
random_split = np.arange(N)
rng.shuffle(random_split)
train_idxs = random_split[:-6042]
test_idxs = random_split[-6042:]
train_ds = ConvmorphDatasetArc(cm_ds, train_idxs)
test_ds = ConvmorphDatasetArc(cm_ds, test_idxs)
train_eval_ds = ConvmorphDatasetArc(cm_ds, train_idxs[-len(test_ds):])
print(len(train_idxs), len(test_idxs), len(train_eval_ds))

69915 6042 6042


In [16]:
with open("drive/MyDrive/LangOn/convmorph/split_inidices_char4.pkl", "wb") as fout:
  pickle.dump({"train_idxs": train_idxs, "test_idxs": test_idxs}, fout)

## Prepare Dataset

In [17]:
from dataclasses import dataclass
import torch.nn.functional as F

@dataclass
class ConvmorphArcModelOutput:
  loss: torch.tensor
  pred_vec: torch.tensor

class ConvmorphArcModel(nn.Module):
  def __init__(self, dropout=0.1):
    super().__init__()

    self.conv1 = nn.Conv2d(1, 128, 3)
    self.norm1 = nn.LayerNorm([128, 18, 18])
    self.conv2 = nn.Conv2d(128, 128, 3)
    self.norm2 = nn.LayerNorm([128, 16, 16])    
    self.conv3 = nn.Conv2d(128, 128, 3)
    self.norm3 = nn.LayerNorm([128, 14, 14])    
    self.conv4 = nn.Conv2d(128, 128, 3)
    self.norm4 = nn.LayerNorm([128, 12, 12])
    self.conv5 = nn.Conv2d(128, 128, 3)
    self.norm5 = nn.LayerNorm([128, 10, 10])
    
    self.pool1 = nn.MaxPool2d(2)
    self.fn1 = nn.Linear(128*5*5, 3000)
    self.drop1 = nn.Dropout(p=dropout)
    # self.fn2 = nn.Linear(512, 200)   
    # self.fn2 = nn.Linear(2048, 1024)
    # self.drop2 = nn.Dropout(p=dropout)
    # self.fn3 = nn.Linear(1024, 512)
    # self.drop3 = nn.Dropout(p=dropout)
    # self.fn4 = nn.Linear(512, 200)
    self.fn4 = nn.Linear(3000, 200)
  
  def forward(self, inputX, target=None, **kwargs):
    
    z = F.relu(self.norm1(self.conv1(inputX)), inplace=True)
    z = F.relu(self.norm2(self.conv2(z)), inplace=True)
    z = F.relu(self.norm3(self.conv3(z)), inplace=True)
    z = F.relu(self.norm4(self.conv4(z)), inplace=True)    
    z = F.relu(self.norm5(self.conv5(z)), inplace=True)    
    z = self.pool1(z).view(-1, 128*5*5)
    o = self.drop1(torch.tanh(self.fn1(z)))    
    # o = self.drop2(torch.tanh(self.fn2(o)))
    # o = self.drop3(torch.tanh(self.fn3(o)))
    o = self.fn4(o)
    pred_vec = o

    if target is not None:
      loss_fct = nn.MSELoss()
      loss = loss_fct(pred_vec, target)
      return ConvmorphArcModelOutput(loss, pred_vec)
    else:      
      return ConvmorphArcModelOutput(float('nan'), pred_vec)
    

In [18]:
def compute_accuracy(data_loader, test_embs):
  n_correct = 0
  n_items = 0
  model.eval()
  for batch_x in tqdm(data_loader):
      with torch.no_grad():
        batch_x = {k: v.to("cuda") for k, v in batch_x.items()}
        word_ids = batch_x["word_id"].cpu().numpy()
        pred_vec = model(**batch_x).pred_vec      
        preds = torch.argmax(torch.matmul(pred_vec, test_embs.transpose(1, 0)), dim=1).cpu().numpy()
        n_correct += np.array(word_ids==preds, dtype=np.int32).sum()
        n_items += len(preds)
  return n_correct/n_items

In [19]:
from torch.optim.lr_scheduler import LambdaLR
# From https://github.com/huggingface/transformers/blob/v4.17.0/src/transformers/optimization.py
def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, last_epoch=-1):    
    def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)

In [20]:
eval_embs = torch.tensor(np.vstack([x["target"] for x in test_ds]), dtype=torch.float32).to("cuda")
train_eval_embs = torch.tensor(np.vstack([x["target"] for x in train_eval_ds]), dtype=torch.float32).to("cuda")

In [21]:
config = dict(
    lr=5e-4,
    scheduler="linear",
    batch_size=128,
    epochs=100,
    dropout=0.1,
    note='128x5x5->fn:3000,200'
)

run_name = datetime.now(pytz.timezone('Asia/Taipei')).strftime("%m%d-%H%M-arc")
wandb.init(project="convmorph", name=run_name, config=config, save_code=True)

wandb: Currently logged in as: seantyh (use `wandb login --relogin` to force relogin)


In [22]:
from itertools import islice
epochs = config["epochs"]
lr = config["lr"]
batch_size = config["batch_size"]

##   Dataset
## -----------
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

##    Model
## ----------
model = ConvmorphArcModel(dropout=config["dropout"]).to("cuda")
optimizer = optim.AdamW(model.parameters(), lr=lr)
total_steps = epochs*len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, 0, total_steps)

torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
random.seed(123)

train_eploss_vec = []
eval_eploss_vec = []

iter_counter = 0
pbar = tqdm(total=total_steps)
for epoch_i in range(epochs):
  train_loss_vec = []
  eval_loss_vec = []
  model.train()  
  pbar.set_description(f"Epoch {epoch_i}/Train")
  for batch_x in train_loader:
    pbar.update(1)
    optimizer.zero_grad()  
    batch_x = {k: v.to("cuda") for k, v in batch_x.items()}
    loss = model(**batch_x).loss
    loss.backward()
    optimizer.step()
    scheduler.step()
    train_loss_vec.append(loss.item())    
    if iter_counter % 100 == 0:
      wandb.log({"train/loss": loss.item()}, step=iter_counter)
    iter_counter += 1
  train_eploss_vec.append(np.mean(train_loss_vec))  

  pbar.set_description(f"Epoch {epoch_i}/Eval")
  for batch_x in eval_loader:
    model.eval()
    with torch.no_grad():
      batch_x = {k: v.to("cuda") for k, v in batch_x.items()}
      loss = model(**batch_x).loss
      eval_loss_vec.append(loss.item())    
  eval_eploss_vec.append(np.mean(eval_loss_vec))  
  train_acc = compute_accuracy(DataLoader(train_eval_ds, batch_size=128, shuffle=False), train_eval_embs)
  test_acc = compute_accuracy(DataLoader(test_ds, batch_size=128, shuffle=False), eval_embs)
  wandb.log({
      "train/epoch-loss": train_eploss_vec[-1],
      "eval/epoch-loss": eval_eploss_vec[-1],
      "eval/train-acc": train_acc,
      "eval/test-acc": test_acc,
      "trainer/lr": scheduler.get_last_lr()[0]
  }, step=iter_counter)
  print(f"train/eval loss: {train_eploss_vec[-1]}/{eval_eploss_vec[-1]}")
wandb.finish()
pbar.close()

  0%|          | 0/54700 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

train/eval loss: 0.00973683074235889/0.004768426539764429


  0%|          | 0/48 [00:00<?, ?it/s]

...

train/eval loss: 0.0016207365998896128/0.0022168905998114496


  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

train/eval loss: 0.0016172144998723072/0.0022164896169366934



eval/epoch-loss,█▇▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/test-acc,▁▁▅▆▇▇▇█████████████████████████████████
eval/train-acc,▁▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
train/epoch-loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁
trainer/lr,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
eval/epoch-loss,0.00222
eval/test-acc,0.738
eval/train-acc,0.93959
train/epoch-loss,0.00162
train/loss,0.00163


In [24]:
import os
out_dir = "drive/MyDrive/LangOn/convmorph/" + run_name
os.makedirs(out_dir, exist_ok=True)
torch.save(model.state_dict(), out_dir+"/model.pth")